In [1]:
# LIBRARY IMPORTS

# Import necessary libraries for building and training the CNN model
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np

In [2]:
# DATASET DIRECTORY CONFIGURATION

# Define the paths to the training and testing datasets
# Ensure the dataset is properly downloaded and unzipped before running the code
test_dir = r"C:\Users\USER\OneDrive\Desktop\Codes\cats-and-dogs\test"
train_dir = r"C:\Users\USER\OneDrive\Desktop\Codes\cats-and-dogs\train"

In [3]:
# IMAGE PARAMETERS

# Define the image size for resizing and the batch size for data loading
IMG_SIZE = (128, 128)  # Resize all images to 128x128 pixels
BATCH_SIZE = 32  # Number of images to process in a batch

In [4]:
# DATA PREPROCESSING & AUGMENTATION

# Apply data augmentation to the training dataset to improve model generalization
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=15,  # Randomly rotate images by up to 15 degrees
    width_shift_range=0.1,  # Randomly shift images horizontally by 10%
    height_shift_range=0.1,  # Randomly shift images vertically by 10%
    horizontal_flip=True,  # Randomly flip images horizontally
    validation_split=0.2  # Reserve 20% of training data for validation
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Only normalize test data

# Create data generators for training, validation, and testing datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',  # Use the training subset
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',  # Use the validation subset
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False  # Do not shuffle test data to maintain order
)

Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [5]:
# SIMPLE CNN MODEL ARCHITECTURE

# Define the initial learning rate for the optimizer
initial_learning_rate = 0.001

# Use ExponentialDecay for dynamic learning rate adjustment
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the Adam optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Define the CNN model with added regularization and dropout layers to reduce overfitting
model = models.Sequential([
    # First convolutional layer with L2 regularization
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),  # Dropout to prevent overfitting

    # Second convolutional layer with L2 regularization
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    # Third convolutional layer with L2 regularization
    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.5),

    # Flatten the feature maps and add dense layers
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.5),

    # Output layer for binary classification
    layers.Dense(1, activation='sigmoid')
])

c:\Users\USER\OneDrive\Desktop\Codes\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# CONFIGURE THE MODEL

# Compile the model with the Adam optimizer, binary crossentropy loss, and accuracy metric
model.compile(
    optimizer=optimizer,  # Use the optimizer with learning rate scheduling
    loss='binary_crossentropy',  # Loss function for binary classification
    metrics=['accuracy']  # Track accuracy during training
)

In [7]:
# TRAINING THE CNN

# Retrain the model with the updated architecture and regularization
history = model.fit(
    train_generator,
    epochs=10,  # Set the number of epochs for training
    validation_data=val_generator
)

Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.5225 - loss: 2.6100 - val_accuracy: 0.5000 - val_loss: 1.5510
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 39s 777ms/step - accuracy: 0.4931 - loss: 1.2577 - val_accuracy: 0.5000 - val_loss: 1.0595
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 43s 858ms/step - accuracy: 0.4850 - loss: 0.9646 - val_accuracy: 0.5000 - val_loss: 0.8901
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 17s 331ms/step - accuracy: 0.5006 - loss: 0.8475 - val_accuracy: 0.5000 - val_loss: 0.8125
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 17s 341ms/step - accuracy: 0.5025 - loss: 0.7911 - val_accuracy: 0.5000 - val_loss: 0.7698
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 15s 293ms/step - accuracy: 0.4944 - loss: 0.7555 - val_accuracy: 0.5000 - val_loss: 0.7432
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 309ms/step - accuracy: 0.5075 - loss: 0.7349 - val_accuracy: 0.5000 - val_loss: 0.7278
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 15s 299ms/step - accuracy: 0.4787 - loss: 0.7230 - val_accurac

In [8]:
# EVALUATE THE MODEL

# Evaluate the trained model on the test dataset
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 16s 514ms/step - accuracy: 0.5370 - loss: 0.7088
Test Loss: 0.7088
Test Accuracy: 0.5370


In [9]:
# SAVE THE MODEL

# Save the trained model to a file for future use
model.save('exercise_6_custom_arostique.h5')
print("Model saved as 'exercise_6_custom_arostique.h5'")

Model saved as 'exercise_6_custom_arostique.h5'


In [10]:
# SIMPLE INFERENCE SCRIPT

from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='exercise_6_trained_model_improved.h5'):
    
    # Predict the class of an image using the trained model.

    # Load the trained model
    model = tf.keras.models.load_model(model_path)

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Make a prediction
    pred = model.predict(img_array)[0, 0]
    label = "Dog" if pred >= 0.5 else "Cat"
    print(f"Prediction: {label} (confidence: {pred:.2f})")

In [11]:
# EXAMPLE USAGE

# Test the model with example images
predict_image(r"C:\Users\USER\OneDrive\Desktop\Codes\cats-and-dogs\run\run_1.jpg")
predict_image(r"C:\Users\USER\OneDrive\Desktop\Codes\cats-and-dogs\run\run_2.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
Prediction: Cat (confidence: 0.17)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
Prediction: Dog (confidence: 0.62)
